# Datasources #

This section explains datasets, dataset specs, and datasource specs.


## Dataset ##

In Synthorus, a dataset is an object providing access to reference data for training a PGM.

Logically, a dataset is a table of data where columns represent random variables and each row represents an _instance_ (also known as a _sample_, _record_, _row_, _datapoint_, or _joint states_). Each instance in a dataset has an instance weight. An instance weight represents a _weight of evidence_. The value of an instance weight is notionally 1, but can be other values to represent multiple or fractional evidence.

Within Synthorus, a dataset is used to construct cross-tables, where a cross-table represent an empirical joint probability distribution over some random variables. Cross-tables are used to build probabilistic graphical models (PGMs) that characterise the statistics of a dataset.

## DatasetSpec ##

Specifying access to a dataset in Synthorus is done using a `DatasetSpec`. A dataset spec is a Pydantic object, and hence is JSON serializable. (Similar to a `SimulatorSpec` for specifying a simulator.) A dataset spec usually contains information to find and load source data. However, some dataset specs make it possible to directly "inline" data into a `DatasetSpec` object. Other `DatasetSpec` objects permit data to be computationally created using functions.

## DatasourceSpec ##

Specifying a datasource in Synthorus is done using a `DatasourceSpec`. A datasource spec is a Pydantic object, and hence is JSON serializable. A datasource spec is composed of a `DatasetSpec` plus other information to support privacy protection and statistical integrity.

Here is an example datasource spec from the `synthorus_demos` package. The example uses CSV formatted data, provided inline. The data spec is show here as JSON.

In [1]:
from synthorus.model.datasource_spec import DatasourceSpec
from synthorus_demos.dataset.example_datasource import make_datasource_spec_acx

datasource: DatasourceSpec = make_datasource_spec_acx()

print(datasource.model_dump_json(indent=2))

{
  "sensitivity": 1.0,
  "rvs": [
    "A",
    "C",
    "X"
  ],
  "dataset_spec": {
    "type": "csv",
    "weight": null,
    "rv_map": null,
    "rv_define": {},
    "input": {
      "type": "inline",
      "inline": "A,C,X\ny,n,n\nn,n,y\ny,n,y\ny,y,y\ny,y,n\nn,y,y\ny,y,y\nn,n,y\nn,n,n\ny,n,n\n"
    },
    "sep": ",",
    "header": true,
    "skip_blank_lines": true,
    "skip_initial_space": false
  },
  "non_distribution_rvs": []
}


Every datasource spec will have the following attributes.
- A _sensitivity_, which is a non-negative number which captures the privacy risk associated with the dataset. It is a Differential Privacy parameter that quantifies how much a query's result can change if one person's data is added or removed. If a dataset contains no private information its sensitivity is zero. If person can have at most one record in the dataset, its sensitivity is one.
- _rvs_, which is a list of random variable names provided by the dataset. That is, it's columns.
- _non_distribution_rvs_, which is a subset of _rvs_ that should not be considered as providing a probability distribution. This is explained in detail below.
- _dataset_spec_, which specifies how to obtain the data (and instance weights).

Here we make a dataset from the datasource spec.

In [2]:
from synthorus.dataset import Dataset

dataset: Dataset = datasource.dataset()

We can now interrogate the dataset for basic metadata.

Here are the dataset random variables and number of records.

In [3]:
dataset.rvs

('A', 'C', 'X')

In [4]:
dataset.number_of_records()

10

We can interrogate what values are in the dataset for each random variable.

In [5]:
print('A', dataset.value_set('A'))
print('C', dataset.value_set('C'))
print('X', dataset.value_set('X'))

A ['y' 'n']
C ['n' 'y']
X ['n' 'y']


A cross-table is represented by a Pandas DataFrame with the requested random variables as columns and the last column being the weight.

Here are some examples:

In [6]:
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


In [7]:
dataset.crosstab(['A', 'C'])

,A,C,
0,n,n,3
1,n,y,1
2,y,n,3
3,y,y,3


In [8]:
dataset.crosstab(['C'])

,C,
0,n,6
1,y,4


The cross-table with no random variables gives the total weight of a dataset.

In [9]:
dataset.crosstab([])

,
0,10


## Types of Datasets ##

The types of dataset available to Synthorus are:
- CSV file or inline text (with generalised separators and line spacing)
- Table Builder file or inline text (as created by the [Australian Bureau of Statistics](https://www.abs.gov.au/statistics/microdata-tablebuilder/tablebuilder))
- Pickled Pandas DataFrame object, as a file
- Parquet file
- Feather file
- A database SQL query (using ODBC or Postgres)
- A mathematical function (with a defined domain).

These are accessed using an appropriate `DatasetSpec`. Every dataset spec is a Pydantic object, and hence is JSON serializable. Below is a detailed description of the different dataset spec classes.

The following examples will use files provided by `synthorus_demos`. The root directories of the files is given by `synthorus_demos.files.ROOT_DIR`.

## DatasetSpecCsv - inline and file ##

A `DatasetSpecCsv` object is a `DatasetSpec` that provides access to a dataset using text comma-separated values (CSV).

The input text can be provided directly to the `DatasetSpecCsv` object or may be read from a file, specified using the file's path.

Here is a CSV dataset spec where the dataset text is provided inline with the spec object.


In [10]:
from synthorus.model.dataset_spec_impl import DatasetSpecCsv, TextInputSpecInline

dataset_spec = DatasetSpecCsv(
    input=TextInputSpecInline(
        inline="""
            A,C,X
            y,n,n
            n,n,y
            y,n,y
            y,y,y
            y,y,n
            n,y,y
            y,y,y
            n,n,y
            n,n,n
            y,n,n
        """
    )
)

# get the dataset
dataset: Dataset = dataset_spec.dataset()

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


For the next example, we use a provided demo file.

The `head` function returns the first few lines of a text file. We use it here to show the demo file.

In [11]:
from synthorus_demos.utils.file_helper import head
from synthorus_demos.demo_files import DATASET_FILES

head(DATASET_FILES / 'acx.csv')

A,C,X
y,n,n
n,n,y
y,n,y
y,y,y


The following shows how a CSV file can be loaded as a dataset.

The `location` argument is the file name. We call the `dataset` method providing a list of directories to search for the file.

In [12]:
from synthorus.model.dataset_spec_impl import TextInputSpecLocation

dataset_spec = DatasetSpecCsv(
    input=TextInputSpecLocation(location='acx.csv')
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES])

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])


,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


A `DatasetSpecCsv` can also be used when other column separators are required.

This demo file is tab separated.

In [13]:
head(DATASET_FILES / 'acx.tsv')

A	C	X
y	n	n
n	n	y
y	n	y
y	y	y


Loading is the same as before, but a separator is specified.

In [14]:
dataset_spec = DatasetSpecCsv(
    input=TextInputSpecLocation(location='acx.tsv'),
    sep='\t',
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES])

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


## DatasetSpecTableBuilder ##

A Table Builder data file is similar to a CSV file but with included meta-data. See the [Australian Bureau of Statistics](https://www.abs.gov.au/statistics/microdata-tablebuilder/tablebuilder) for more details.

Here is an example.

In [15]:
head(DATASET_FILES / 'table_builder' / 'age-sex[NSW].csv', number_of_lines=20)

Australian Bureau of Statistics

"2021 Census - employment, income and education"
"AGEP Age and SEXP Sex by STATE (UR)"
"Counting: Person Records"

Filters:
"Default Summation","Person Records"
"STATE (UR)","New South Wales"


"AGEP Age","SEXP Sex",
"0","Male",47069,
,"Female",44443,
"1","Male",47785,
,"Female",45457,
"2","Male",48413,
,"Female",45172,
"3","Male",48218,
,"Female",45414,


Here we load and use this file using a `DatasetSpecTableBuilder` spec.


In [16]:
from synthorus.model.dataset_spec_impl import DatasetSpecTableBuilder

dataset_spec = DatasetSpecTableBuilder(
    input=TextInputSpecLocation(location='age-sex[NSW].csv')
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES / 'table_builder'])

# show the random variables
dataset.rvs

('AGEP Age', 'SEXP Sex')

Here is a cross-table showing there were more females than males in the census.

In [17]:
dataset.crosstab(['SEXP Sex'])

,SEXP Sex,
0,Female,4087975
1,Male,3984174


## DatasetSpecPickle ##

This example uses a pickled Pandas DataFrame.

Before using the pickle file in Synthorus we show it here using the Pandas `read_pickle` function.

In [18]:
import pandas as pd

with (DATASET_FILES / 'acx.pkl').open('rb') as f:
    df = pd.read_pickle(f)
    print(df)

   A  C  X
0  y  n  n
1  n  n  y
2  y  n  y
3  y  y  y
4  y  y  n
5  n  y  y
6  y  y  y
7  n  n  y
8  n  n  n
9  y  n  n


A pickled Pandas dataframe is access using a `DatasetSpecPickle` spec.

In [19]:
from synthorus.model.dataset_spec_impl import DatasetSpecPickle

dataset_spec = DatasetSpecPickle(
    location='acx.pkl'
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES])

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


## DatasetSpecParquet ##

This example uses a Parquet file.

Before using the Parquet file in Synthorus we show it here using the Pandas `read_parquet` function.


In [20]:
with (DATASET_FILES / 'acx.parquet').open('rb') as f:
    df = pd.read_parquet(f)
    print(df)

   A  C  X
0  y  n  n
1  n  n  y
2  y  n  y
3  y  y  y
4  y  y  n
5  n  y  y
6  y  y  y
7  n  n  y
8  n  n  n
9  y  n  n


A pickled Parquet file is access using a `DatasetSpecParquet` spec.


In [21]:
from synthorus.model.dataset_spec_impl import DatasetSpecParquet

dataset_spec = DatasetSpecParquet(
    location='acx.parquet'
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES])

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


## DatasetSpecFeather ##

This example uses a Feather file.

Before using the Parquet file in Synthorus we show it here using the Pandas `read_feather` function.


In [22]:
with (DATASET_FILES / 'acx.feather').open('rb') as f:
    df = pd.read_feather(f)
    print(df)

   A  C  X
0  y  n  n
1  n  n  y
2  y  n  y
3  y  y  y
4  y  y  n
5  n  y  y
6  y  y  y
7  n  n  y
8  n  n  n
9  y  n  n


A pickled Feather file is access using a `DatasetSpecFeather` spec.


In [23]:
from synthorus.model.dataset_spec_impl import DatasetSpecFeather

dataset_spec = DatasetSpecFeather(
    location='acx.feather'
)

# get the dataset
dataset: Dataset = dataset_spec.dataset([DATASET_FILES])

# demo using the dataset
dataset.crosstab(['A', 'C', 'X'])

,A,C,X,
0,n,n,n,1
1,n,n,y,2
2,n,y,y,1
3,y,n,n,2
4,y,n,y,1
5,y,y,n,1
6,y,y,y,2


## DatasetSpecFunction ##

A "function" dataset spec let you define a dataset as a function over a domain.

The domain is defined using one or more random variables, each with a list of possible states. The possible states of a random variable is defined either explicitly or as an integer `n` which is shorthand for `list(range(n))`.

The function is a Python expression with identifiers for the input random variables. You can think of this as the body of a lambda function.

In the example below, the function domain is a single random variable, "q", with possible values [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].

In [24]:
from synthorus.model.dataset_spec_impl import DatasetSpecFunction

dataset_spec = DatasetSpecFunction(
    output_rv='double_q',
    function='q * 2',
    rvs={'q': 10},  # the 'q' domain is list(range(10))
)

# get the dataset
dataset: Dataset = dataset_spec.dataset()

# show the dataset random variables
dataset.rvs

('q', 'double_q')

Here is the data defined by the function.

In [25]:
dataset.crosstab(['q', 'double_q'])

,q,double_q,
0,0,0,1
1,1,2,1
2,2,4,1
3,3,6,1
4,4,8,1
5,5,10,1
6,6,12,1
7,7,14,1
8,8,16,1
9,9,18,1


## DatasetSpecDBMS - Postgres ##

A `DatasetSpecDBMS` spec creates a dataset from an SQL query of a database. Synthorus cas use two APIs, one for `postgres

This example uses the [MindsDB](https://docs.mindsdb.com/integrations/sample-database) public demonstration Postgres database.

In this example, credentials to access the database are provided explicitly in the dataset spec (as a `connection` dictionary).
Below we show how credentials can be provided outside the Synthorus system using "local configuration". Local
configuration can be provided in a `config.py` file or through operating system environment variables.

In [26]:
from synthorus.model.dataset_spec_impl import DatasetSpecDBMS

dataset_spec = DatasetSpecDBMS(
    type='postgres',
    connection={
        'user': 'demo_user',
        'password': 'demo_password',
        'host': 'samples.mindsdb.com',
        'dbname': 'demo',
    },
    schema_name='demo',
    table_name='home_rentals',
)

# get the dataset
dataset: Dataset = dataset_spec.dataset()

# show the random variables
dataset.rvs

('number_of_rooms',
 'number_of_bathrooms',
 'sqft',
 'location',
 'days_on_market',
 'neighborhood',
 'rental_price',
 'created_at')

Here is an example cross-table.

In [27]:
dataset.crosstab(['neighborhood'])

,neighborhood,count
0,alcatraz_ave,71
1,westbrae,93
2,downtown,135
3,berkeley_hills,50
4,south_side,63
5,thowsand_oaks,92


## DatasetSpecDBMS - ODBC ##

Synthorus can use arbitrary ODBC drivers that you may have installed. To create an ODBC dataset, you can do something like this.

```
from synthorus.model.dataset_spec_impl import DatasetSpecDBMS

example_dataset_spec = DatasetSpecDBMS(
    type='odbc',
    connection={
        'driver': '{Microsoft Access Driver (*.mdb, *.accdb)}',
        'dbq': r'C:\databases\my_database.accdb',
        'uid': 'demo_user',
        'pwd': 'demo_password',
    },
    schema_name='demo',
    table_name='the_table'
)
```

Here is another example, using local configuration (explained below).

```
example_dataset_spec = DatasetSpecDBMS(
    type='odbc',
    connection={
        'SERVER': None,     # will use config.DB_SERVER
        'DATABASE': None,   # will use config.DB_DATABASE
        'UID': None,        # will use config.DB_UID
        'PORT': 1433,
        'AUTHENTICATION': 'ActiveDirectoryInteractive',
        'DRIVER': '{ODBC Driver 17 for SQL Server}',
    },
    schema_name='demo',
    table_name='the_table'
)
```


## Connection Local Configuration ###

In many situations some information should not be directly specified a `connection` dictionary. For example usernames and passwords. If the value of an entry in a `connection` dictionary is `None`, then Synthorus will resolve the value using "local configuration" when the dataset is created. E.g., if a connection entry with key "PASSWORD" has value `None`, Synthorus will use local configuration variable "DB_PASSWORD". In general, an entry "_k_: `None`" will be resolved using local configuration variable "`DB_`_k_".

Local configuration is managed by the module `synthorus.utils.config_help`. For a given config variable, `config_help` will first look for a module called `config`, i.e., a Python file called `config.py` in your Python path, to see if the variable is defined there. If not, then `config_help` checks the operating system environment variables.

Assuming `config.py` contains:
```
X = 2
Y = 'yes'
```

And the operating system environment contains:
```
X=1
Z=abc
```

The following will all be true:
```
from synthorus.utils.config_help import config


'X' in config
'Y' in config
'Z' in config
'A' not in config

config.X == 2
config['Y'] == 'yes'
config.get('Z') == 'abc'
config.get('A') is None
config.get('A', 'no') == 'no'

len(config) == 3
```
